In [1]:
import numpy as np
import pandas as pd

from eva import EVStation
from eva import DictAttr

In [2]:
from tqdm import tqdm
import sys
import time
import timeit

In [3]:
sys.version

'3.9.12 | packaged by conda-forge | (main, Mar 24 2022, 23:24:38) \n[Clang 12.0.1 ]'

In [4]:
evs_config = {'N': 50000, 'Ns': 20,
              'Tagc': 4.0, 'socf': 0.2,
              'agc': 0, 'ict': 0,
              'memory_save': 1,
              'seed': 2022, 'r': 0.5}

mcs_config = {'ts': 18.0, 'h': 1.0,
              'no_tqdm': 0, 'th': 1.0}

nd_param = {'soci': {'mu': 0.3, 'var': 0.05, 'lb': 0.2, 'ub': 0.4},
            'socd': {'mu': 0.8, 'var': 0.03, 'lb': 0.7, 'ub': 0.9},
            'ts1': {'mu': -6.5, 'var': 3.4, 'lb': 0.0, 'ub': 5.5},
            'ts2': {'mu': 17.5, 'var': 3.4, 'lb': 5.5, 'ub': 24.0},
            'tf1': {'mu': 8.9, 'var': 3.4, 'lb': 0.0, 'ub': 20.9},
            'tf2': {'mu': 32.9, 'var': 3.4, 'lb': 20.9, 'ub': 24.0},
            'tt': {'mu': 0.5, 'var': 0.02, 'lb': 0, 'ub': 1}}

ud_param = {'Pc': {'lb': 5.0, 'ub': 7.0},
            'Pd': {'lb': 5.0, 'ub': 7.0},
            'nc': {'lb': 0.88, 'ub': 0.95},
            'nd': {'lb': 0.88, 'ub': 0.95},
            'Q': {'lb': 20.0, 'ub': 30.0}}


In [5]:
import logging
logger = logging.getLogger()
logging.getLogger("eva").setLevel(logging.INFO)

In [6]:
evs = EVStation(name='EVS1',
                mcs_config=mcs_config,
                config=evs_config,
                ud_param=ud_param,
                nd_param=nd_param)

Memory save is turned on, EVs out of time range [18.0, 19.0] are dropped.


In [10]:
evs.MCS.data.s.as_df()['soc0']

0        3.309851
1        3.198377
2        4.015586
3        3.011465
4        3.688535
           ...   
17368    0.733367
17369    1.871234
17370    0.358874
17371    0.875062
17372    0.255674
Name: soc0, Length: 17373, dtype: float64

In [7]:
evs.MCS.data.d.as_df()

,idx,u,u0,soc,c,lc,sx,na,ama,agc,mod
0,203,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,625,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,803,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,858,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1050,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
17368,49995,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17369,49996,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17370,49997,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17371,49998,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
sdf = evs.MCS.data.s.as_df()
sdf[sdf['tf'] < 18]

In [ ]:
self = evs
dsp = self.MCS.data.s  # pointer to static data
ddp = self.MCS.data.d  # pointer to dynamic data

mask_u1 = dsp.get(src='ts') > (self.MCS.config.ts + self.MCS.config.th)
mask_u2 = dsp.get(src='tf') < self.MCS.config.ts
mask = mask_u1 | mask_u2
drop_id = np.where(mask)[0]

for ary in [self.idx, self.MCS.data.idx, self.MCS.data.s.v, self.MCS.data.d.v]:
    ary = np.delete(ary, drop_id, axis=0)

getattr(self.MCS.data.s, 'idx')
setattr(self.MCS.data.s, 'idx', np.delete(self.MCS.data.s.idx, drop_id, axis=0))

In [ ]:
ary.shape

In [ ]:
np.logical_and(mask_u1, mask_u2)

In [ ]:
mask_u1

In [ ]:
mask_u2

In [ ]:
mask_u1 & mask_u2

In [ ]:
evs.MCS.data.d.v.shape

In [ ]:
evs.MCS.data.d.set('soc', cond=np.array([False]*49999 + [True]),
                   data=np.array([1, 2]))
evs.MCS.data.d.as_df()

In [ ]:
evs.MCS.data.s.get('Pc', cond=np.array([True]*5))

In [ ]:
evs.MCS.data.d.as_df()

In [ ]:
evs.MCS.config.ts

In [ ]:
dsp = evs.MCS.data.s.v
mask = dsp[:, getattr(evs.MCS.data.col.s, 'ts')] < 0.3
dsp[mask & mask].shape

In [ ]:
evs.MCS.data.s.as_df()

In [ ]:

dc = {'s': ['ts', 'tf', 'tt', 'soc0', 'na0',
            'soci', 'socd', 'Pc', 'Pd',
            'nc', 'nd', 'Q'],
      'd': ['u', 'u0', 'soc', 'c', 'lc', 'sx',
            'na', 'ama', 'agc', 'mod']}
{
's': {c: dc['s'].index(c) for c in dc['s']},
'd': {c: dc['d'].index(c) for c in dc['d']}
}

from eva import DictAttr
sd = DictAttr({c: dc['s'].index(c) for c in dc['s']})
dd = DictAttr({c: dc['d'].index(c) for c in dc['d']})
dcd = DictAttr({'s': sd, 'd': dd})

In [ ]:
np.empty(100, 100)

In [ ]:
np.arange(50000).shape

In [ ]:
%%timeit -r 100 -n 10
dcd.s.Pc

In [ ]:
dcd.d.u0

In [ ]:
xdf = evs.MCS.data.d.copy()
xsf = evs.MCS.data.s.copy()

In [ ]:
%%timeit -r 20 -n 20
xdf['u'] * xdf['c'] * xsf['Pc'] * xsf['nc'] * 1/3600

In [ ]:
xdf.columns

In [ ]:
xsf.columns

In [ ]:
xd = evs.MCS.data.d.values
xs = evs.MCS.data.s.values

In [ ]:
%%timeit -r 20 -n 20
xd[:, 1] * xd[:, 4] * xs[:, 8] * xs[:, 10] * 1/3600

In [ ]:
evs = EVStation(name='EVS1',
                mcs_config=mcs_config,
                config=evs_config,
                ud_param=ud_param,
                nd_param=nd_param)

evs.MCS.config.tf = 3600
evs.MCS.config.no_tqdm = 0
evs.MCS.run()
evs.MCS.data
evs.MCS.data.d.info(verbose=False, memory_usage=True)